In [1]:
# python 3
import pandas as pd
import nltk
import re

from nltk.corpus import stopwords
from nltk import ngrams

pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows",3000)


In [2]:
# get majority labels and all labels

def clean_label(df_agent1, df_agent2, df_agent3):
    label_column = df_agent1.filter(regex='Label').columns
    label1, label2, label3, label4, label5 = label_column[0], \
                                             label_column[1], \
                                             label_column[2], \
                                             label_column[3], \
                                             label_column[4]
    
    df_agent1[label_column] = df_agent1[label_column].fillna('null_value')
    df_agent2[label_column] = df_agent2[label_column].fillna('null_value')
    df_agent3[label_column] = df_agent3[label_column].fillna('null_value')
    
    df_agent1['agent1_labels'] = df_agent1.apply(lambda x: set([x[label1],
                                 x[label2],
                                 x[label3],
                                 x[label4],                         
                                 x[label5]]),axis=1)
    
    df_agent1['agent2_labels'] = df_agent2.apply(lambda x: set([x[label1],
                                 x[label2],
                                 x[label3],
                                 x[label4],                         
                                 x[label5]]),axis=1)
    
    df_agent1['agent3_labels'] = df_agent3.apply(lambda x: set([x[label1],
                                 x[label2],
                                 x[label3],
                                 x[label4],                         
                                 x[label5]]),axis=1)
    
    df_agent1['majority_vote'] = df_agent1.apply(lambda x: x['agent1_labels'].intersection(x['agent2_labels'])\
                                         .union(x['agent2_labels'].intersection(x['agent3_labels']))\
                                         .union(x['agent1_labels'].intersection(x['agent3_labels']))\
                               , axis=1)
    
    df_agent1['union_vote'] = df_agent1.apply(lambda x: x['agent1_labels'].union(x['agent2_labels'])\
                                         .union(x['agent3_labels'])\
                               , axis=1)
    
    # trick to associate new variable with column method operations
    mv = df_agent1['majority_vote']
    mv.apply(lambda x: x.discard('null_value'));
    
    uv = df_agent1['union_vote']
    uv.apply(lambda x: x.discard('null_value'));
    return df_agent1.drop(label_column, axis=1)

In [3]:
# get data

one_ic = pd.read_csv('data/1-ic.csv')
one_gc = pd.read_csv('data/1-gc.csv')
one_sc = pd.read_csv('data/1-sc.csv')

two_jm = pd.read_csv('data/2-jm.csv')
two_mg = pd.read_csv('data/2-mg.csv')
two_nb = pd.read_csv('data/2-nb.csv')

three_rs = pd.read_csv('data/3-rs.csv')
three_rt = pd.read_csv('data/3-rt.csv')
three_sj = pd.read_csv('data/3-sj.csv')

tph_batch1 = pd.read_csv('data/tph_batch1.csv')

In [4]:
tph_batch1.rename({'sampled_bid_id': 'bid_id'}, axis='columns', inplace=True)

In [5]:
# clean labels

df1 = clean_label(one_ic, one_gc, one_sc)
df2 = clean_label(two_jm, two_mg, two_nb)
df3 = clean_label(three_rs, three_rt, three_sj)

In [6]:
# merge data

df = df1.append(df2).append(df3)

In [7]:
df = df.merge(tph_batch1[['bid_id','message_timestamp','message']], \
              how='left', on = ['bid_id','message_timestamp'])

In [8]:
#df['is_hire_majority'] = df.apply(lambda x: ('Hire' in x['majority_vote'])*1, axis=1)
#df['is_hire_any'] = df.apply(lambda x: ('Hire' in x['union_vote'])*1, axis=1)

In [9]:
df['is_contact_info'] = df.apply(lambda x: ('Contact Information' in x['majority_vote'])*1, axis=1)
df['is_scheduling'] = df.apply(lambda x: ('Scheduling - Meeting or Job' in x['majority_vote'])*1, axis=1)
df['is_price'] = df.apply(lambda x: ('Price' in x['majority_vote'])*1, axis=1)
df['is_payment'] = df.apply(lambda x: ('Payment' in x['majority_vote'])*1, axis=1)
df['is_generic'] = df.apply(lambda x: ('Generic Answer' in x['majority_vote'])*1, axis=1)

In [10]:
# helper function to dislay frequency of words from a blob of text 
def get_info(txt):
    words = nltk.tokenize.word_tokenize(txt)

    bigrams = nltk.bigrams(words)
    trigrams = nltk.trigrams(words)

    word_dist = nltk.FreqDist(words)
    top_N = 200
    custom_stopwords = set((u'.', u',', u'?', u'!', u')', u':', u'\'s', u'('))

    words_except_stop_dist = nltk.FreqDist(w for w in words if w not in custom_stopwords) 

    print('Unigram frequencies:')
    rslt = pd.DataFrame(words_except_stop_dist.most_common(top_N),
                        columns=['Word', 'Frequency']).set_index('Word')
    print(rslt)
    print('=' * 60)

    # Bigram Frequencies
    print('Bigram frequencies:')
    bigrams_freq = nltk.FreqDist(bigrams)
    rslt = pd.DataFrame(bigrams_freq.most_common(top_N),
                        columns=['Word', 'Frequency']).set_index('Word')

    print(rslt)
    print('=' * 60)

    # Trigram Frequencies
    print('Trigram frequencies:')
    trigrams_freq = nltk.FreqDist(trigrams)
    rslt = pd.DataFrame(trigrams_freq.most_common(top_N),
                        columns=['Word', 'Frequency']).set_index('Word')

    print(rslt)
    print('=' * 60)


In [11]:
# helper function to dislay precision and recall information from a term and a specific column we evaluate against 

def display_info(term, col):
    pat1 = re.compile("^.*"+term+".*$" , flags = re.DOTALL) # equivalent to str.contains # re.DOTALL applies REGEX to muliple line
    predicted = df['message'].str.lower().str.match(pat1).astype(int)
    precision = df.loc[predicted[predicted==1].index, col].sum()/\
            df.loc[predicted[predicted==1].index, col].count()

    recall = df.loc[predicted[predicted==1].index, col].sum()/\
            df[col].sum()    
        
    false_positive_cases = df.loc[(df['message'].str.lower().str.match(pat1)) \
                              & (df.is_scheduling == 0)]['message']
    
    num_terms = df.loc[predicted[predicted==1].index, col].sum()
    return [term, 
            "{0:.0f}%".format(precision*100), 
            "{0:.0f}%".format(recall*100), 
            num_terms]

In [12]:
# helper function to dislay precision and recall information from a group of terms and a specific column we evaluate against 

def display_info_multiple(key_words, col):
    regex_pattern = "^.*("
    for i,w in enumerate(key_words):
        if i != len(key_words) -1:
            regex_pattern += w +'|'
        else:
            regex_pattern += w 

    regex_pattern += ").*$"
    
    pat1 = re.compile(regex_pattern , flags = re.DOTALL) # equivalent to str.contains
    predicted = df['message'].str.lower().str.match(pat1).astype(int)
    precision = df.loc[predicted[predicted==1].index, col].sum()/\
            df.loc[predicted[predicted==1].index, col].count()

    recall = df.loc[predicted[predicted==1].index, col].sum()/\
            df[col].sum()    
        
    false_positive_cases = df.loc[(df['message'].str.lower().str.match(pat1)) \
                              & (df.is_scheduling == 0)]['message']
    
    num_terms = df.loc[predicted[predicted==1].index, col].sum()
    return [regex_pattern, 
            "{0:.0f}%".format(precision*100), 
            "{0:.0f}%".format(recall*100), 
            num_terms
           ]

In [13]:
# test function
display_info('available', 'is_scheduling')

['available', '91%', '13%', 671]

In [14]:
# test function
display_info_multiple(['available','weekday'], 'is_scheduling')

['^.*(available|weekday).*$', '91%', '13%', 693]

## Scheduling

In [15]:
df[(df.is_scheduling==1) & (df.is_generic == 1)].shape

(88, 18)

In [16]:
df[(df.is_scheduling==1)].shape

(5182, 18)

In [17]:
get_info(df[df.is_scheduling == 1].message.str.lower().str.cat(sep=' '))

Unigram frequencies:
              Frequency
Word                   
you           5457     
i             5224     
to            3996     
the           3374     
and           2593     
a             2449     
for           2435     
can           1780     
we            1682     
is            1577     
be            1528     
have          1386     
me            1352     
if            1307     
at            1283     
that          1248     
in            1234     
would         1232     
will          1209     
on            1187     
your          1172     
do            1171     
it            990      
my            989      
are           980      
time          977      
of            942      
or            905      
this          861      
tomorrow      853      
with          776      
hi            760      
so            740      
work          729      
available     712      
am            702      
call          693      
what          691      
know          686  

                        Frequency
Word                             
(let, me, know)         424      
(., thank, you)         217      
(for, you, ?)           188      
(., i, 'm)              159      
(would, like, to)       153      
(., i, will)            144      
(give, you, a)          144      
(work, for, you)        141      
(are, you, available)   134      
(i, will, be)           134      
(., let, me)            132      
(., i, am)              132      
(be, able, to)          127      
(do, you, have)         125      
(me, know, if)          122      
(thank, you, for)       121      
(for, you, .)           119      
(., if, you)            116      
(please, let, me)       112      
(., i, can)             111      
(works, for, you)       106      
(you, a, call)          105      
(would, you, like)      104      
(., i, have)            99       
(give, me, a)           91       
(i, have, a)            91       
(., do, you)            90       
(i, can, do)  

In [18]:
# TO DO: Tina to investigate more word/phrases here. Put everything you tried in here.
# TO DO: add month, date, time pattern (e.g. "digit am" etc)

schedule_keywords = \
["book", "appointment","schedule","what time",'available', 'availability',\
 'works for you','morning','afternoon','evening', 'tomorrow', 'next week',\
 'monday','tuesday','wednesday','thursday','friday','saturday','sunday','weekday','weekend',\
 'me know when','a call', 'give you a', 'works for you', 'work for you', 'forward to hearing',\
 'today',' am ', ' pm ', 'can you make',\
 's your next availability', 'when i can call you', \
 'are you available on my date', 'when can i give you a call', ' give you a call', \
 'see you at', 'would like to book', 'book you']


In [19]:
scheduling_terms = pd.DataFrame(columns=('Term', 'Precision', 'Recall', 'Count'))
for i, term in enumerate(schedule_keywords):
    scheduling_terms.loc[i] = display_info(term, 'is_scheduling')


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in long_scalars
  


In [20]:
scheduling_terms.sort_values(by=['Precision', 'Recall'], ascending=False)

,Term,Precision,Recall,Count
32,when i can call you,nan%,0%,0
33,are you available on my date,nan%,0%,0
5,availability,98%,2%,127
34,when can i give you a call,98%,1%,43
19,weekday,97%,1%,28
3,what time,96%,4%,211
29,pm,95%,2%,88
13,tuesday,93%,4%,186
14,wednesday,91%,3%,165
4,available,91%,13%,671


In [21]:
# TO DO: Tina to finalize the set of words to use by picking from the scheduling_terms list

scheduling_regex_words = ['availability','available','weekday','what time', 'weekday',\
                          'work for you','works for you', 'schedul','tomorrow','afternoon',\
                         'monday','tuesday','wednesday','thursday','friday','saturday','sunday',\
                         'can you make','when can i give you a call']

In [22]:
display_info_multiple(scheduling_regex_words, 'is_scheduling')

['^.*(availability|available|weekday|what time|weekday|work for you|works for you|schedul|tomorrow|afternoon|monday|tuesday|wednesday|thursday|friday|saturday|sunday|can you make|when can i give you a call).*$',
 '85%',
 '59%',
 3065]

In [23]:
# Example of further investigation
# the left over cases, i.e. 1 - recall cases
pat1 = '^.*(availability|available|weekday|what time|weekday|work for you|works for you|schedul|tomorrow|afternoon|monday|tuesday|wednesday|thursday|friday|saturday|sunday|can you make|when can i give you a call).*$'
df[(df.is_scheduling == 1) & (~df['message'].str.lower().str.match(pat1))].message.head(100)

0      Greetings..\nAre you available for resume writing?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
3      April. Can likely do est this wk & Work maybe the next. let me know your phone # & address.  Tx. Wade                                      

In [24]:
## Example of further investigations
## investigate the regex related to word "book"
pat1 = re.compile('^.*(book).*$' , flags = re.DOTALL) # equivalent to str.contains
pat2 = re.compile('^.*(availability|available|weekday|what time|weekday|work for you|works for you|schedul|tomorrow|afternoonmonday|tuesday|wednesday|thursday|friday|saturday|sunday).*$' , flags = re.DOTALL) # equivalent to str.contains

# df[(df.is_scheduling == 1) & (df['message'].str.lower().str.match(pat1))\
#   & (~df['message'].str.lower().str.match(pat2))].message.head(100)

get_info(df[(df.is_scheduling == 1) & (df['message'].str.lower().str.match(pat1))\
   & (~df['message'].str.lower().str.match(pat2))].message.str.lower().str.cat(sep=' '))

Unigram frequencies:
               Frequency
Word                    
i              90       
the            85       
you            85       
to             75       
for            42       
and            40       
your           32       
book           32       
is             29       
a              28       
we             25       
booked         24       
if             24       
so             23       
in             22       
do             22       
me             21       
as             20       
on             19       
that           18       
my             18       
be             17       
would          17       
it             17       
have           17       
$              17       
will           16       
can            16       
are            15       
let            15       
at             14       
need           13       
thank          13       
'm             13       
date           12       
with           12       
this           12       
abou

## Payment

In [25]:
df[(df.is_payment==1) & (df.is_generic == 1)].shape

(8, 18)

In [26]:
df[(df.is_payment==1)].shape

(437, 18)

In [27]:
get_info(df[df.is_payment == 1].message.str.lower().str.cat(sep=' '))

Unigram frequencies:
           Frequency
Word                
the        758      
you        753      
i          726      
to         626      
and        490      
a          340      
for        328      
is         248      
can        246      
your       232      
we         231      
of         228      
do         222      
will       219      
that       201      
be         198      
in         187      
me         187      
have       184      
it         178      
if         172      
or         169      
on         154      
my         148      
with       144      
would      131      
check      117      
so         116      
payment    113      
at         113      
as         112      
send       106      
deposit    99       
cash       96       
$          91       
pay        90       
are        86       
get        86       
need       81       
like       79       
this       78       
know       76       
work       73       
also       71       
thanks     70

In [28]:
# TO DO: Tina to investigate more word/phrases here. Put everything you tried in here.
# Hint: use the investigation examples to get inspirations 

payment_keywords = \
["pay", "payment", "credit card", "cash", "check", "debit", "invoice", "payment", "deposit", "paypal", \
 "the deposit", "send you", "the contract", "a deposit", "the check", \
 "do you accept", "i will need", "you can pay", "will send you",\
 "send me your", "can send you", "a check", "check or cash",\
 "do i pay", "send you an", "send you the", "deposit to hold the",\
 "do you accept credit", "do you take cash", "check or credit card", "payment do you accept", \
 "at your earliest convenience", "hold the date", "send you an invoice"]


In [29]:
payment_terms = pd.DataFrame(columns=('Term', 'Precision', 'Recall', 'Count'))
for i, term in enumerate(payment_keywords):
    payment_terms.loc[i] = display_info(term, 'is_payment')


In [30]:
payment_terms.sort_values(by=['Precision', 'Recall'], ascending=False)

,Term,Precision,Recall,Count
2,credit card,95%,10%,42
9,paypal,91%,7%,31
14,the check,91%,5%,20
23,do i pay,90%,2%,9
21,a check,89%,9%,41
3,cash,87%,22%,97
10,the deposit,85%,7%,29
1,payment,82%,24%,104
7,payment,82%,24%,104
8,deposit,82%,21%,93


In [31]:
# TO DO: Tina to finalize the set of words to use by picking from the payment_terms list

payment_regex_words = ['debit','credit card','the check','paypal', 'venmo',\
                          'the deposit', 'do i pay']

In [32]:
display_info_multiple(payment_regex_words, 'is_payment')

['^.*(debit|credit card|the check|paypal|venmo|the deposit|do i pay).*$',
 '92%',
 '29%',
 126]